In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import sys
from typing import Dict, List, Optional, Tuple, Union
from collections import Counter, defaultdict, deque

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec15.txt'

In [2]:
WALL = 0
MOVED = 1
OXYGEN = 2
UNSEEN = 3

UP, DOWN, LEFT, RIGHT = 1, 2, 3, 4
OPPOSITE = [None, 2, 1, 4, 3]

class InputWrapper:
    def __init__(self, robot):
        self.robot = robot
        self.stack = [(None, 0, 0)]
    
    def get(self):
        while self.stack:
            for i, direction in enumerate(util.four_ways(self.robot.pos_x, self.robot.pos_y), 1):
                if self.robot.painted[direction] == UNSEEN:
                    self.stack.append((i, direction[0], direction[1]))
                    return i
            
            # If we've been everywhere, go back where we came
            last_dir = self.stack.pop()[0]
            if last_dir is None:
                # We're out of steps! Halt
                return None
            return OPPOSITE[last_dir]

class OutputWrapper:
    def __init__(self, robot, input_wrapper):
        self.robot = robot
        self.input_wrapper = input_wrapper
    
    def __call__(self, val):
        last_dir, to_x, to_y = self.input_wrapper.stack[-1]
        self.robot.painted[(to_x, to_y)] = val
        
        if val == WALL:
            # We're in the same place. Don't step
            self.input_wrapper.stack.pop()
        elif val in [MOVED, OXYGEN]:
            # Open space. Step.
            self.robot.pos_x = to_x
            self.robot.pos_y = to_y
            
class Robot:
    def __init__(self, ops):
        self.ops = ops
        self.pos_x = 0
        self.pos_y = 0
        self.painted = defaultdict(lambda: UNSEEN)
        self.painted[(0, 0)] = MOVED

    def simulate(self):
        input_wrapper = InputWrapper(self)
        output_wrapper = OutputWrapper(self, input_wrapper)
        sim.simulate(self.ops, inputs=input_wrapper, output_func=output_wrapper)
    
    def printme(self):
        min_x = min(key[0] for key in self.painted.keys())
        max_x = max(key[0] for key in self.painted.keys())
        min_y = min(key[1] for key in self.painted.keys())
        max_y = max(key[1] for key in self.painted.keys())
        for y in range(min_y, max_y + 1):
            for x in range(min_x, max_x + 1):
                if self.painted[(x, y)] == MOVED:
                    print('.', end='')
                elif self.painted[(x, y)] == WALL:
                    print('#', end='')
                elif self.painted[(x, y)] == OXYGEN:
                    print('O', end='')
                else:
                    print('~', end='')
            print()        

In [3]:
# Explore the space with DFS
robot = Robot(sim.read_ops(INPUT_PATH.read_text().strip()))
robot.simulate()

In [4]:
robot.printme()

~###~###########~###########~#####~#####~
#...#...........#...........#.....#.....#
#.#.#######.#####.#####.###.###.#.#####.#
#.#.#.....#.#...#.#...#...#.....#.......#
#.#.#.###.#.#.#.#.###.###.######~######.#
#.#...#O#.#.#.#...#...#...#.....#.......#
#.#####.#.#.#.#####.#.#.#.#.###.#.######~
#.....#.#.#.#.......#.#.#.#.#...#.#.....#
#.###.#.#.#.#.#######.#.###.#.###.#.###.#
#.#...#...#...#.....#.#.#...#.#.....#.#.#
#.#.###.#####.#.###.###.#.###.#######.#.#
#.#...#.....#.#.#.......#...#.....#...#.#
#.###.#####.###.#######.###.#####.###.#.#
#...#.....#.....#.....#.#...#...#...#...#
~######.#.#.#####.###.###.#####.###.#.##~
#.....#.#.#.#.....#.#.....#.......#.#.#.#
#.###.###.###.#####.#######.#######.#.#.#
#.#.......#...#.............#.......#...#
#.#.#######.###.#.#########.#.######~##.#
#.#...#...#.#...#.#.....#.#.#...#...#...#
#.###.#.#.#.#.#.###.###.#.#.###.#.###.##~
#.#.#.#.#...#.#...#...#.#.....#.#.#...#.#
#.#.#.#.#####.###.#####.#######.#.#.###.#
#.#...#...#.....#.....#.........#.

In [5]:
# Then get distances with BFS
def dist(
      painted: Dict[Tuple[int, int], int],
      start: Tuple[int, int] = (0, 0),
      stop: Optional[Tuple[int, int]] = None
    ) -> Union[int, Dict[Tuple[int, int], int]]:
    """
    Return the distance to stop from start if present, else return
    all pairs distances from start.
    
    painted is structured as in the Robot class above
    """
    queue = deque([(0, start)])  # (Distance from start, (x, y))
    seen = {start: 0}  # (x, y): distance from start
    while queue:
        dist, (x, y) = queue.popleft()
        for to in util.four_ways(x, y):
            if stop and to == stop:
                return dist + 1
            if to not in seen and painted[to] != WALL:
                seen[to] = dist + 1
                queue.append((dist + 1, to))
    return seen

In [6]:
ox = [key for key, val in robot.painted.items() if val == OXYGEN][0]
print(f'The answer to part 1 is {dist(robot.painted, stop=ox)}')

The answer to part 1 is 242


In [7]:
print(f'The answer to part 2 is {max(dist(robot.painted, start=ox).values())}')

The answer to part 2 is 276
